## Query to Dataframe

## Dependencies

In [7]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from config import (ServerName, UserName, Password, port, DataBase)
import psycopg2

## Connection

In [8]:
rds_connection_string = f'{UserName}:{Password}@{ServerName}:{port}/{DataBase}'
engine = create_engine(f'postgresql://{rds_connection_string}')

## Function to Query the Database

In [9]:
# "sqlfile": contains the file.sql with the query and 
# "json_file": It contains the route "folder/name.json" for the output
# "json_file": It contains the route "folder/name.csv" for the output
def run_sql(sqlfile, json_file, csv_file):

    fd = open(sqlfile, 'r')
    sqlFile = fd.read()
    fd.close()

    # all SQL commands (split on ';')
    sqlCommands = sqlFile.split(';')    

    try:
        # connect to the PostgreSQL server
        engine = create_engine(f'postgres+psycopg2://{UserName}:{Password}@{ServerName}:{port}/{DataBase}')
        conn = engine.connect()
        # create table one by one
        for command in sqlCommands:
            trans = conn.begin()
            conn.execute(command)
            trans.commit()
        # Close connection
        tables = engine.table_names()
        conn.close()
        SQL_Query = pd.read_sql_query(command, con=engine)
        # Create Dataframe
        df = pd.DataFrame(SQL_Query)
        # Create CSV File
        df.to_csv(csv_file, index=False)
        # Create JSON file
        json = df.to_json(orient='records')
        f = open(json_file,"w")
        f.write( str(json) )
        f.close()
        print('The Dataframe and json file were created')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

In [10]:
sql_file = 'queries/full_query.sql'
csv_file = '../output_charts/full_query.csv'
json_file = '../output_charts/full_query.json'
run_sql(sql_file, json_file, csv_file)

The Dataframe and json file were created
